In [1]:
from sqlalchemy import ForeignKey, create_engine, ForeignKey, String, Integer, Column
from sqlalchemy.orm import sessionmaker, declarative_base
import uuid

In [2]:
Base = declarative_base()

In [3]:
def generate_uuid():
    return str(uuid.uuid4())

In [4]:
class Users(Base):
    __tablename__ = "users"
    userID = Column("userID", String, primary_key=True, default=generate_uuid)
    firstName = Column('firstName', String)
    lastName = Column('lastName', String)
    profileName = Column('profileName', String)
    email = Column('email', String)
    
    def __init__(self, firstName, lastName, profileName, email):
        self.firstName = firstName
        self.lastName = lastName
        self.profileName = profileName
        self.email = email
        

In [5]:
db = "sqlite:///socialDB.db"
engine = create_engine(db)
Base.metadata.create_all(bind=engine)

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
firstName = "Edward"
lastName = "Clark"
profileName = "Ed7812"
email = "ed@mail.com"

In [8]:
user = Users(firstName, lastName, profileName, email)
session.add(user)
session.commit()
print(user.userID)
print("user added")

a904ec94-a92d-4f5a-86f3-2b9b48d14eff
user added


In [9]:
class Posts(Base):
    __tablename__ = "posts"
    postID = Column("postID", String, primary_key=True, default=generate_uuid)
    postContent = Column("postContent", String)
    userID = Column("userID", String, ForeignKey("users.userID"))

    def __init__(self, userID, postContent):
        self.userID = userID
        self.postContent = postContent

In [10]:
Base.metadata.create_all(bind=engine)

In [11]:
post = Posts(userID=user.userID, postContent='This is a post')  # Use the userID from the user you just added
session.add(post)
session.commit()

print("post added")
print(post.postContent, post.postID)

post added
This is a post ca8fc23a-74b1-4f11-a029-2bbb425c8a86
